<a href="https://colab.research.google.com/github/AakifY03/YDP/blob/main/YDP%C3%B6dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

# Google Drive'ı bağlayın
from google.colab import drive
drive.mount('/content/drive')

# Veri setinin yolunu tanımlayın
data_path = '/content/drive/My Drive/plates/'

# Yolun varlığını kontrol edin
if os.path.exists(data_path):
    print(f"Veriler bu dizinde mevcut: {data_path}")
else:
    print("Veriler bulunamadı. Lütfen yolu kontrol edin.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Veriler bu dizinde mevcut: /content/drive/My Drive/plates/


In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data_dir="/content/drive/My Drive/plates/"
os.listdir(data_dir)

['sample_submission.csv', 'test', 'train', '.DS_Store']

In [ ]:
import shutil
from tqdm import tqdm

train_dir = 'train'
val_dir = 'val'

class_names = ['cleaned', 'dirty']

for dir_name in [train_dir, val_dir]:
    for class_name in class_names:
        os.makedirs(os.path.join(dir_name, class_name), exist_ok=True)

for class_name in class_names:
    source_dir = os.path.join(data_dir, 'train', class_name)
    for i, file_name in enumerate(tqdm(os.listdir(source_dir))):
        if i % 6 != 0:
            dest_dir = os.path.join(train_dir, class_name)
        else:
            dest_dir = os.path.join(val_dir, class_name)
        shutil.copy(os.path.join(source_dir, file_name), os.path.join(dest_dir, file_name))

100%|██████████| 21/21 [00:05<00:00,  3.94it/s]


In [ ]:
import torch
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import time
import copy

from torchvision import transforms, models
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_dataset = torchvision.datasets.ImageFolder(train_dir, train_transforms)
val_dataset = torchvision.datasets.ImageFolder(val_dir, val_transforms)

batch_size = 8
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=batch_size)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, num_workers=batch_size)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
len(train_dataloader), len(train_dataset)

(5, 34)

In [ ]:
from torchvision import models


model = models.swin_t(weights=models.Swin_T_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/swin_t-704ceda3.pth" to /root/.cache/torch/hub/checkpoints/swin_t-704ceda3.pth
100%|██████████| 108M/108M [00:00<00:00, 143MB/s] 


In [10]:
import torch.nn as nn

num_classes = len(train_dataset.classes)
in_features = model.head.in_features
model.head = nn.Linear(in_features, num_classes)

In [11]:
# Modeli cihaz (GPU/CPU) üzerine taşıyalım
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Kayıp fonksiyonu ve optimizasyonu ayarlayalım
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Eğitim fonksiyonu
def train_model(model, train_dataloader, criterion, optimizer, num_epochs=10):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()  # Eğitim moduna al
        running_loss = 0.0
        corrects = 0
        total = 0

        for inputs, labels in train_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            # İleriye doğru geçiş
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Geriye doğru geçiş
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = corrects.double() / total

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}")

        # En iyi modeli kaydet
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = copy.deepcopy(model.state_dict())

    model.load_state_dict(best_model_wts)
    return model


In [12]:
# Modeli eğitin
model = train_model(model, train_dataloader, criterion, optimizer, num_epochs=25)


Epoch 1/25, Loss: 1.0930, Accuracy: 0.5588
Epoch 2/25, Loss: 1.0997, Accuracy: 0.3235
Epoch 3/25, Loss: 1.2394, Accuracy: 0.5000
Epoch 4/25, Loss: 1.0261, Accuracy: 0.5588
Epoch 5/25, Loss: 0.8991, Accuracy: 0.5294
Epoch 6/25, Loss: 0.7795, Accuracy: 0.5294
Epoch 7/25, Loss: 0.9104, Accuracy: 0.4118
Epoch 8/25, Loss: 1.2379, Accuracy: 0.4118
Epoch 9/25, Loss: 0.9249, Accuracy: 0.4412
Epoch 10/25, Loss: 0.6647, Accuracy: 0.6176
Epoch 11/25, Loss: 1.3063, Accuracy: 0.4412
Epoch 12/25, Loss: 0.7889, Accuracy: 0.4706
Epoch 13/25, Loss: 0.8464, Accuracy: 0.5588
Epoch 14/25, Loss: 0.7116, Accuracy: 0.4706
Epoch 15/25, Loss: 0.7642, Accuracy: 0.5000
Epoch 16/25, Loss: 0.8415, Accuracy: 0.4706
Epoch 17/25, Loss: 0.8103, Accuracy: 0.3529
Epoch 18/25, Loss: 0.7241, Accuracy: 0.5294
Epoch 19/25, Loss: 0.7377, Accuracy: 0.4118
Epoch 20/25, Loss: 0.8088, Accuracy: 0.4706
Epoch 21/25, Loss: 0.7430, Accuracy: 0.4706
Epoch 22/25, Loss: 0.8351, Accuracy: 0.5000
Epoch 23/25, Loss: 0.6963, Accuracy: 0.47

In [13]:
def evaluate_model(model, val_dataloader):
    model.eval()  # Değerlendirme moduna al
    corrects = 0
    total = 0

    with torch.no_grad():  # No gradient calculation, sadece tahmin
        for inputs, labels in val_dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            corrects += torch.sum(preds == labels.data)
            total += labels.size(0)

    accuracy = corrects.double() / total
    print(f"Validation Accuracy: {accuracy:.4f}")

# Değerlendirmeyi başlat
evaluate_model(model, val_dataloader)


Validation Accuracy: 0.5000
